In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [2]:
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        #self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(inplace=True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(inplace=True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(inplace=True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(inplace=True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(inplace=True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(inplace=True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1, y2, y3, y4], 1)    
        

In [3]:
class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
        )

        self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
        self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)

        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception(480, 192,  96, 208, 16,  48,  64)
        self.b4 = Inception(512, 160, 112, 224, 24,  64,  64)
        self.c4 = Inception(512, 128, 128, 256, 24,  64,  64)
        self.d4 = Inception(512, 112, 144, 288, 32,  64,  64)
        self.e4 = Inception(528, 256, 160, 320, 32, 128, 128)

        self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(kernel_size=8, stride=1)
        self.linear = nn.Linear(1024, 10)

    def forward(self, inputs):
        network = self.pre_layers(inputs)
        network = self.a3(network)
        network = self.b3(network)
        network = self.maxpool(network)
        network = self.a4(network)
        network = self.b4(network)
        network = self.c4(network)
        network = self.d4(network)
        network = self.e4(network)
        network = self.maxpool(network)
        network = self.a5(network)
        network = self.b5(network)
        network = self.avgpool(network)
        network = network.view(network.size(0), -1)
        out = self.linear(network)
        return out, network

In [4]:
GoogLeNet = GoogLeNet()
print(GoogLeNet)

GoogLeNet(
  (pre_layers): Sequential(
    (0): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (a3): Inception(
    (b1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (b2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
    )
    (b3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

# 测试，可视化网络结构

In [5]:
class Inception_self(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        #下一个Inception的输入通道数为：n1x1 + n3x3 + n5x5 + pool_planes
        #self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
        super(Inception_self, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(inplace=True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(inplace=True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(inplace=True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(inplace=True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(inplace=True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(inplace=True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1, y2, y3, y4], 1) 
class GoogLeNet_self(nn.Module):
    def __init__(self):
        super(GoogLeNet_self, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=True),
        )

        self.a3 = Inception_self(20,  7,  10, 13, 2, 4, 4)
        self.b3 = Inception_self(28, 13, 13, 20, 4, 10, 7)

        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception_self(50, 20,  1, 21, 2,  5,  7)
        self.b4 = Inception_self(53, 16, 12, 23, 3,  7,  7)
        self.c4 = Inception_self(53, 13, 13, 26, 3,  7,  7)
        self.d4 = Inception_self(53, 12, 15, 29, 4,  7,  7)
        self.e4 = Inception_self(55, 26, 17, 32, 4, 13, 13)
        self.a5 = Inception_self(84, 26, 17, 32, 4, 13, 13)
        self.b5 = Inception_self(84, 39, 20, 39, 5, 13, 13)


        self.avgpool = nn.AvgPool2d(kernel_size=8, stride=1)
        self.linear = nn.Linear(2600, 10)

    def forward(self, inputs):
        network = self.pre_layers(inputs)
        
        network = self.a3(network)
        network = self.b3(network) 
        network = self.maxpool(network)
        
        network = self.a4(network)
        network = self.b4(network)
        network = self.c4(network)
        network = self.d4(network)
        network = self.e4(network)
        network = self.maxpool(network)
        
        network = self.a5(network)
        network = self.b5(network)
        network = self.avgpool(network)
        
        network = network.view(network.size(0), -1)
        out = self.linear(network)
        return out, network

In [6]:
GoogLeNet_self = GoogLeNet_self()
print(GoogLeNet_self)

GoogLeNet_self(
  (pre_layers): Sequential(
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (a3): Inception_self(
    (b1): Sequential(
      (0): Conv2d(20, 7, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (b2): Sequential(
      (0): Conv2d(20, 10, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(10, 13, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
    )
    (b3): Sequential(
      (0): Conv2d(20, 2, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=T

In [7]:
GoogLeNet_self(Variable(torch.randn(2,3,48,48)))

(tensor([[ 0.0506,  0.1443,  0.1272,  0.2046, -0.0199, -0.0190,  0.0551,  0.0431,
          -0.2143, -0.0954],
         [-0.0246,  0.2795,  0.0553,  0.1534,  0.1458,  0.1141,  0.0512,  0.1574,
          -0.2940, -0.2754]], grad_fn=<ThAddmmBackward>),
 tensor([[0.3154, 0.3542, 0.2884,  ..., 0.0103, 0.0125, 0.0127],
         [0.5570, 0.6767, 0.6979,  ..., 0.3337, 0.3033, 0.2523]],
        grad_fn=<ViewBackward>))

In [8]:
dummy_input = Variable(torch.randn(2,3,48,48))
with SummaryWriter(comment='GoogLeNet_self') as w:
    w.add_graph(GoogLeNet_self,(dummy_input, ))